In [ ]:
# Install awscli and boto3 before this lab


import boto3
import datetime
import json
client = boto3.client('cloudwatch')

response = client.describe_alarm_history(
#     AlarmName='Owen_CloudWatchAlarm',                 
    HistoryItemType='ConfigurationUpdate',
    MaxRecords=1
)
print(response)


In [ ]:
import boto3
import json
import requests
import time

client = boto3.client('cloudwatch')

# 取得 Metric Name -> 'OK'|'ALARM'|'INSUFFICIENT_DATA'
def get_alarm_metric_name(alarm_count):
    metric_name = response['MetricAlarms'][alarm_count]['MetricName']
    return metric_name

# 取得 instance id 
def get_alarm_instance_id(alarm_count , disk_alarm_count=0): # 判斷 disk 指標請把 disk_alarm_count 參數修改為 1
    instance_id = response['MetricAlarms'][alarm_count]['Dimensions'][disk_alarm_count]['Value']
    return instance_id

# 發送 Slack notification
def send_notification_to_slack(channel,payload_instanc_id,payload_metric_name,):
    payload_header = {'Content-type': 'application/json'}
    dest_url = "your slack webhook url"
    payload = {
        "text": '機器ID: \n' + payload_instanc_id + '\n'+ '\n指標為: ' + payload_metric_name + '\n' + '已經超過 80% 囉',
        "channel": channel
    }
    json_payload = json.dumps(payload)
    result = requests.post(dest_url, data=json_payload, headers=payload_header)
    return result
    
# Check MetricAlarms 
response = client.describe_alarms(
    StateValue='ALARM',
    MaxRecords=10,
)
if response['MetricAlarms'] == []:
    print('There is no alarm')
    print(len(response['MetricAlarms']))
    send_notification_to_slack("#devops","無任何機器","無任何指標")
else:
    total_alarm_count = len(response['MetricAlarms']) # total_alarm_count = 總共有多少 alarm
    for all_count in range(total_alarm_count): 
        if get_alarm_metric_name(all_count) == "MemoryUtilization":
            result = '機器: ' + get_alarm_instance_id(all_count) + ' 記憶體使用量已達 40%'
            print(result)
            send_notification_to_slack("#devops",get_alarm_instance_id(all_count),get_alarm_metric_name(all_count))
        elif get_alarm_metric_name(all_count) == "DiskSpaceUtilization":
            result = '機器: ' + get_alarm_instance_id(all_count,1) + ' 硬碟使用量已達 40%'
            print(result)
            send_notification_to_slack("#devops",get_alarm_instance_id(all_count,1),get_alarm_metric_name(all_count))
        else:
            print('Not found memory and disk metric')

In [1]:
# 改 code 因應 aws lambda 會 cache 執行結果的 data 問題
import boto3
import json
import requests
import time

# Initialize 
client = boto3.client('cloudwatch')
payload_header = {'Content-type': 'application/json'}
dest_url = "your slack webhook url "


time.sleep(10)
response = client.describe_alarms(
    StateValue='ALARM',
    MaxRecords=10,
)
if response['MetricAlarms'] == []:
    send_notification_to_slack("#devops","No Instance id","No metric")
else:
    total_alarm_count = len(response['MetricAlarms']) # count MetricAlarms
    mem_alarm_list = []
    disk_alarm_list = []
    for all_count in range(total_alarm_count):
        if response['MetricAlarms'][all_count]['MetricName'] == "MemoryUtilization":
            result = '機器: ' + response['MetricAlarms'][all_count]['Dimensions'][0]['Value'] +  " 記憶體使用量已達 40%\n"
            mem_alarm_list.append(response['MetricAlarms'][all_count]['Dimensions'][0]['Value'])
            print(result)
            payload = {
                "text": result,
                "channel":'#devops'
            }
            json_payload = json.dumps(payload)
            result = requests.post(dest_url, data=json_payload, headers=payload_header)
        elif response['MetricAlarms'][all_count]['MetricName'] == "DiskSpaceUtilization":
            result = '機器: ' + response['MetricAlarms'][all_count]['Dimensions'][1]['Value'] + " 硬碟使用量已達 40%\n"
            disk_alarm_list.append(response['MetricAlarms'][all_count]['Dimensions'][1]['Value'])
            print(result)
            payload = {
                "text": result,
                "channel":'#devops'
            }
            json_payload = json.dumps(payload)
            result = requests.post(dest_url, data=json_payload, headers=payload_header)
        else:
            print('Not found memory and disk metric')
        payload = {
            "text" : '`' + str(mem_alarm_list) + '`' + ' 記憶體使用量達 40%\n'+ '`' + str(disk_alarm_list) + '`' + ' 硬碟使用量達 40%\n',
            "channel" : '#devops'
        }
    json_payload = json.dumps(payload)
    result = requests.post(dest_url , data=json_payload,headers=payload_header)

機器: i-0dad6adf01b9fb5ee 硬碟使用量已達 40%

機器: i-02d995d47c807ae65 硬碟使用量已達 40%

機器: i-02d995d47c807ae65 記憶體使用量已達 40%



In [ ]:
# 這是原本有 cache 問題的 lambda function code 

# Initials aws resources connection
client = boto3.client('cloudwatch')
response = client.describe_alarms(
    StateValue='ALARM',
    MaxRecords=10,
)
    # describe_alarm
# 取得 Metric Name -> 'OK'|'ALARM'|'INSUFFICIENT_DATA'
def get_alarm_metric_name(alarm_count):
    metric_name = response['MetricAlarms'][alarm_count]['MetricName']
    return metric_name

# 取得 instance id 
def get_alarm_instance_id(alarm_count , disk_alarm_count=0): # 判斷 disk 指標請把 disk_alarm_count 參數修改為 1
    instance_id = response['MetricAlarms'][alarm_count]['Dimensions'][disk_alarm_count]['Value']
    return instance_id

# 發送 Slack notification
def send_notification_to_slack(channel,payload_instanc_id,payload_metric_name,):
    payload_header = {'Content-type': 'application/json'}
    dest_url = "your slack webhook url"
    payload = {
        "text": '機器ID: \n' + payload_instanc_id + '\n'+ '\n指標為: ' + payload_metric_name + '\n' + '已經超過 80% 囉',
        "channel": channel
    }
    json_payload = json.dumps(payload)
    result = requests.post(dest_url, data=json_payload, headers=payload_header)
    return result

def lambda_handler(event, context):
    time.sleep(10)
    # TODO implement
    if response['MetricAlarms'] == []:
        print('There is no alarm')
        print(len(response['MetricAlarms']))
        send_notification_to_slack("#devops","無任何機器","無任何指標")
    else:
        total_alarm_count = len(response['MetricAlarms']) # total_alarm_count = 總共有多少 alarm
        for all_count in range(total_alarm_count): 
            if get_alarm_metric_name(all_count) == "MemoryUtilization":
                result = '機器: ' + get_alarm_instance_id(all_count) + ' 記憶體使用量已達 40%'
                print(result)
                send_notification_to_slack("#devops",get_alarm_instance_id(all_count),get_alarm_metric_name(all_count))
            elif get_alarm_metric_name(all_count) == "DiskSpaceUtilization":
                result = '機器: ' + get_alarm_instance_id(all_count,1) + ' 硬碟使用量已達 40%'
                print(result)
                send_notification_to_slack("#devops",get_alarm_instance_id(all_count,1),get_alarm_metric_name(all_count))
            else:
                print('Not found memory and disk metric')
    return {
        'statusCode': 200,
        'body': json.dumps('lambda function is ok!')
        
    }

In [ ]:
# 修改後上 lambda 的 code
import boto3
import json
import requests
import time

# Initialize
client = boto3.client('cloudwatch')
payload_header = {'Content-type': 'application/json'}
dest_url = "your slack webhook url"

def lambda_handler(event, context):
    time.sleep(10)
    response = client.describe_alarms(
       StateValue='ALARM',
       MaxRecords=10,
       )
    if response['MetricAlarms'] == []:
        result = " No any machine in alarm status"
        paylaod = {
            "text" : result,
            "channel" : "#devops"
            
        }
        json_payload = json.dumps(payload)
        result = requests.post(dest_url, data=json_payload, headers=payload_header)
    else:
        total_alarm_count = len(response['MetricAlarms']) # count MetricAlarms
        for all_count in range(total_alarm_count):
            if response['MetricAlarms'][all_count]['MetricName'] == "MemoryUtilization":
                result = '機器: ' + response['MetricAlarms'][all_count]['Dimensions'][0]['Value'] +  " 記憶體使用量已達 40%\n"
                print(result)
                payload = {
                    "text": result,
                    "channel":'#devops'
                    }
                json_payload = json.dumps(payload)
                result = requests.post(dest_url, data=json_payload, headers=payload_header)
            elif response['MetricAlarms'][all_count]['MetricName'] == "DiskSpaceUtilization":
                result = '機器: ' + response['MetricAlarms'][all_count]['Dimensions'][1]['Value'] + " 硬碟使用量已達 40%\n"
                print(result)
                payload = {
                    "text": result,
                    "channel":'#devops'
                    }
                json_payload = json.dumps(payload)
                result = requests.post(dest_url, data=json_payload, headers=payload_header)
            else:
                print('Not found memory and disk metric')
    return {
        'statusCode': 200,
        'body': json.dumps('lambda function is ok!')
        
    }

In [ ]:
# 優化回傳的訊息 , 因應環境如果有多個 alarm，傳送到 slack channel 的時候將訊息變得較可讀，不然會太亂

# 修改後上 lambda 的 code
import boto3
import json
import requests
import time

# Initialize
client = boto3.client('cloudwatch')
payload_header = {'Content-type': 'application/json'}
dest_url = "your slack webhook url"

def lambda_handler(event, context):
    time.sleep(10)
    response = client.describe_alarms(
       StateValue='ALARM',
       MaxRecords=10,
       )
    if response['MetricAlarms'] == []:
        result = " No any machine in alarm status"
        paylaod = {
            "text" : result,
            "channel" : "#devops"
            
        }
        json_payload = json.dumps(payload)
        result = requests.post(dest_url, data=json_payload, headers=payload_header)
    else:
        total_alarm_count = len(response['MetricAlarms']) # count MetricAlarms
        mem_alarm_id_list = []
        disk_alarm_id_list = []
        for all_count in range(total_alarm_count):
            if response['MetricAlarms'][all_count]['MetricName'] == "MemoryUtilization":
                result = '機器: ' + response['MetricAlarms'][all_count]['Dimensions'][0]['Value'] +  " 記憶體使用量已達 40%\n"
                mem_alarm_id_list.append(response['MetricAlarms'][all_count]['Dimensions'][0]['Value'])
            elif response['MetricAlarms'][all_count]['MetricName'] == "DiskSpaceUtilization":
                result = '機器: ' + response['MetricAlarms'][all_count]['Dimensions'][1]['Value'] + " 硬碟使用量已達 40%\n"
                disk_alarm_id_list.append(response['MetricAlarms'][all_count]['Dimensions'][1]['Value'])
            else:
                print('Not found memory and disk metric')
        payload = {
            "text" : '`' + str(mem_alarm_id_list) + '`' + ' 記憶體使用量達 40%\n'+ '`' + str(disk_alarm_id_list) + '`' + ' 硬碟使用量達 40%\n',
            "channel" : '#devops'
        }
    json_payload = json.dumps(payload)
    result = requests.post(dest_url , data=json_payload,headers=payload_header)
    return {
        'statusCode': 200,
        'body': json.dumps('lambda function is ok!')
        
    }